In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import gkey

In [2]:
# read weather data into a dataframe

weather_df = pd.read_csv('output_data/cities.csv')
weather_df.head()

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Date
0,Saskylakh,RU,71.9167,114.0833,-4.76,89,10.40,23,1618080067
1,Yulara,AU,-25.2406,130.9889,64.40,48,11.50,0,1618080093
2,Qinhuangdao,CN,39.9317,119.5883,48.07,62,6.44,71,1618080302
3,Nikolskoye,RU,59.7035,30.7861,37.00,86,2.24,40,1618080054
4,Ranong,TH,9.9658,98.6348,78.30,85,4.38,58,1618080492


In [3]:
# configure gmaps with API key

gmaps.configure(api_key = gkey)

weather_df.dtypes

City                object
Country             object
Latitude           float64
Longitude          float64
Max Temperature    float64
Humidity             int64
Wind Speed         float64
Cloudiness           int64
Date                 int64
dtype: object

In [7]:
# store latitude and longitude as location

locations = weather_df[['Latitude', 'Longitude']]


,Humidity
0,89.0
1,48.0
2,62.0
3,86.0
4,85.0
...,...
528,24.0
529,60.0
530,52.0
531,93.0


In [8]:
# create a humidity heatmap layer



fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = weather_df['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# new data frame with weather criteria

low_temp = 50
high_temp = 75
hum_lvl = 70
cloud = 75

perfect_weather = weather_df.loc[(weather_df['Max Temperature'] >= low_temp) & 
                                 (weather_df['Max Temperature'] <= high_temp) &
                                 (weather_df['Humidity'] >= hum_lvl) & 
                                 (weather_df['Cloudiness'] >= cloud)
                                ]

In [24]:
# limit options for sanity

vacation_spots = perfect_weather.sample(n=5)
vacation_spots['Hotel'] = ''
vacation_spots

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Date,Hotel
16,Hermanus,ZA,-34.4187,19.2345,60.01,89,2.24,100,1618080043,
354,Gadsden,US,34.0143,-86.0066,72.00,73,8.05,75,1618080581,
329,Esperance,AU,-33.8667,121.9000,57.20,88,10.36,100,1618080074,
415,Whitianga,NZ,-36.8333,175.7000,66.99,99,5.39,80,1618080597,
35,Ancud,CL,-41.8697,-73.8203,57.99,82,10.36,75,1618080033,


In [26]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
radius = 5000

for index, row in vacation_spots.iterrows():
    lat = vacation_spots.loc[index, 'Latitude']
    lng = vacation_spots.loc[index, 'Longitude']
    params = {
        'location' : f'{lat}, {lng}',
        'type' : 'hotel',
        'radius' : radius,
        'key' : gkey,
    }
    response = requests.get(base_url, params = params).json()
    vacation_spots.loc[index, 'Hotel'] = response['results'][1]['name']
    
vacation_spots

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Date,Hotel
16,Hermanus,ZA,-34.4187,19.2345,60.01,89,2.24,100,1618080043,Misty Waves Boutique Hotel
354,Gadsden,US,34.0143,-86.0066,72.00,73,8.05,75,1618080581,Hampton Inn Gadsden
329,Esperance,AU,-33.8667,121.9000,57.20,88,10.36,100,1618080074,Hospitality Esperance
415,Whitianga,NZ,-36.8333,175.7000,66.99,99,5.39,80,1618080597,Beachside Resort Whitianga
35,Ancud,CL,-41.8697,-73.8203,57.99,82,10.36,75,1618080033,Hotel Arena Gruesa
